In [1]:
import os
import joblib
import numpy as np
import pandas as pd

In [2]:
data_folder = './data'
df = pd.read_csv(os.path.join(data_folder, 'prep_test.csv'))
df.shape

(1783345, 54)

In [3]:
regressor = joblib.load('./models/2_neural_net.pkl')
regressor

C:\Users\bruno\Anaconda3\lib\site-packages\sklearn\base.py:310: UserWarning: Trying to unpickle estimator StandardScaler from version 1.0.2 when using version 0.24.1. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
C:\Users\bruno\Anaconda3\lib\site-packages\sklearn\base.py:310: UserWarning: Trying to unpickle estimator MLPRegressor from version 1.0.2 when using version 0.24.1. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
C:\Users\bruno\Anaconda3\lib\site-packages\sklearn\base.py:310: UserWarning: Trying to unpickle estimator Pipeline from version 1.0.2 when using version 0.24.1. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


Pipeline(steps=[('standardscaler', StandardScaler()),
                ('mlpregressor',
                 MLPRegressor(hidden_layer_sizes=(40, 20, 10),
                              random_state=7))])

In [21]:
y_pred = regressor.predict(df)
y_pred.shape

(1783345, 5)

In [22]:
target_columns = [
    'nu_nota_lc',
    'nu_nota_ch',
    'nu_nota_cn',
    'nu_nota_mt',
    'nu_nota_redacao'
]

In [17]:
df_test = pd.read_csv(os.path.join(data_folder, 'test.csv'), usecols=['NU_INSCRICAO', 'TP_PRESENCA_CN', 'TP_PRESENCA_CH', 'TP_PRESENCA_LC', 'TP_PRESENCA_MT', 'TP_STATUS_REDACAO'])
df_test.head()

,NU_INSCRICAO,TP_PRESENCA_CN,TP_PRESENCA_CH,TP_PRESENCA_LC,TP_PRESENCA_MT,TP_STATUS_REDACAO
0,5d5b362b-7388-4ac6-81b3-23573e4e2d3a,0,0,0,0,NaN
1,52356efd-3239-4cd2-a444-416625dfc560,1,1,1,1,1.0
2,1ba42e9a-dd61-4405-9b08-79b728ad23c9,1,1,1,1,1.0
3,16fa0f84-a88f-43e6-bcbb-8d5ea41e5f03,1,1,1,1,1.0
4,fa663d11-5052-4ab2-b771-3a3de3bdec55,1,1,1,1,1.0


In [18]:
df_test.isna().sum()

NU_INSCRICAO              0
TP_PRESENCA_CN            0
TP_PRESENCA_CH            0
TP_PRESENCA_LC            0
TP_PRESENCA_MT            0
TP_STATUS_REDACAO    409802
dtype: int64

In [19]:
df_test['TP_STATUS_REDACAO'] = df_test['TP_STATUS_REDACAO'].fillna(0)

In [23]:
df_pred = pd.DataFrame(data=y_pred, columns=[c.upper() for c in target_columns])
df_pred.describe()

,NU_NOTA_LC,NU_NOTA_CH,NU_NOTA_CN,NU_NOTA_MT,NU_NOTA_REDACAO
count,1.783345e+06,1.783345e+06,1.783345e+06,1.783345e+06,1.783345e+06
mean,5.217763e+02,5.096157e+02,4.782514e+02,5.200521e+02,5.739425e+02
std,3.495157e+01,4.255279e+01,4.170356e+01,6.252117e+01,8.438451e+01
min,4.080550e+02,4.009019e+02,3.799966e+02,3.811247e+02,2.294500e+02
25%,4.965091e+02,4.782292e+02,4.469828e+02,4.731230e+02,5.216369e+02
50%,5.177017e+02,5.016768e+02,4.683003e+02,5.045720e+02,5.621603e+02
75%,5.435147e+02,5.334538e+02,4.997510e+02,5.525057e+02,6.161572e+02
max,6.435661e+02,6.841307e+02,6.348929e+02,7.759007e+02,8.813980e+02


In [24]:
df_pred.loc[df_test['TP_PRESENCA_LC'] != 1, 'NU_NOTA_LC'] = 0
df_pred.loc[df_test['TP_PRESENCA_CH'] != 1, 'NU_NOTA_CH'] = 0
df_pred.loc[df_test['TP_PRESENCA_CN'] != 1, 'NU_NOTA_CN'] = 0
df_pred.loc[df_test['TP_PRESENCA_MT'] != 1, 'NU_NOTA_MT'] = 0
df_pred.loc[df_test['TP_STATUS_REDACAO'] != 1, 'NU_NOTA_REDACAO'] = 0
df_pred.describe()

,NU_NOTA_LC,NU_NOTA_CH,NU_NOTA_CN,NU_NOTA_MT,NU_NOTA_REDACAO
count,1.783345e+06,1.783345e+06,1.783345e+06,1.783345e+06,1.783345e+06
mean,4.030806e+02,3.937197e+02,3.501709e+02,3.823660e+02,4.323372e+02
std,2.223219e+02,2.183365e+02,2.168229e+02,2.396582e+02,2.652915e+02
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,4.649640e+02,4.487671e+02,0.000000e+00,0.000000e+00,0.000000e+00
50%,5.065524e+02,4.887179e+02,4.535224e+02,4.839319e+02,5.409527e+02
75%,5.365451e+02,5.242522e+02,4.887262e+02,5.369073e+02,6.020769e+02
max,6.405200e+02,6.841307e+02,6.348929e+02,7.759007e+02,8.813980e+02


In [25]:
df_submission = df_test.join(df_pred).drop(columns=['TP_PRESENCA_CN', 'TP_PRESENCA_CH', 'TP_PRESENCA_LC', 'TP_PRESENCA_MT', 'TP_STATUS_REDACAO'])
df_submission.head()

,NU_INSCRICAO,NU_NOTA_LC,NU_NOTA_CH,NU_NOTA_CN,NU_NOTA_MT,NU_NOTA_REDACAO
0,5d5b362b-7388-4ac6-81b3-23573e4e2d3a,0.000000,0.000000,0.000000,0.000000,0.000000
1,52356efd-3239-4cd2-a444-416625dfc560,573.815418,565.555627,532.116832,590.616825,722.941374
2,1ba42e9a-dd61-4405-9b08-79b728ad23c9,540.854777,528.494109,500.018080,548.420040,628.782482
3,16fa0f84-a88f-43e6-bcbb-8d5ea41e5f03,556.094278,554.426727,520.631035,595.904716,613.951444
4,fa663d11-5052-4ab2-b771-3a3de3bdec55,558.077786,558.761156,530.753124,606.569177,659.912803


In [26]:
df_submission.to_csv('submissions/3_pred.csv', index=False)